# Using the torchcam algorithm library

## Importing the Toolkit

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from PIL import Image
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


In [6]:
!rm -rf torch-cam # if you have.

In [8]:
# Install torch-cam
!git clone https://github.com/frgfm/torch-cam.git 
!pip install -e torch-cam/.

正克隆到 'torch-cam'...
remote: Enumerating objects: 6167, done.
remote: Counting objects: 100% (1705/1705), done.
remote: Compressing objects: 100% (623/623), done.
remote: Total 6167 (delta 1188), reused 1550 (delta 1072), pack-reused 4462
接收对象中: 100% (6167/6167), 9.63 MiB | 3.83 MiB/s, 完成.
处理 delta 中: 100% (4250/4250), 完成.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///home/featurize/%E5%9B%BE%E5%83%8F%E5%88%86%E7%B1%BB/6-%E5%8F%AF%E8%A7%A3%E9%87%8A%E6%80%A7%E5%88%86%E6%9E%90%E3%80%81%E6%98%BE%E8%91%97%E6%80%A7%E5%88%86%E6%9E%90/1.torch-cam%E5%B7%A5%E5%85%B7%E5%8C%85%EF%BC%9ACAM%E7%83%AD%E5%8A%9B%E5%9B%BE/torch-cam
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Running setup.py develop for torchcam


In [ ]:
#Restart the kernel

In [1]:
import torchcam

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from PIL import Image
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


In [4]:
model = torch.load('checkpoint/(18-1-91).pth')#Import the trained classification model.
model = model.eval().to(device)

## Import CAM methods

In [16]:
from torchcam.methods import SSCAM
cam_extractor = SSCAM(model)

In [17]:
from torchvision import transforms  #Image preprocessing: scaling, cropping, conversion to Tensor, normalisation
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## Generate heatmaps

In [18]:
import os
from torchcam.utils import overlay_mask

In [19]:
!find . -iname '.ipynb_checkpoints'#Delete redundant documents

find: paths must precede expression: `redundant'


In [20]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

In [21]:
!find . -iname '.ipynb_checkpoints'#Look again for redundant files.

In [24]:
img_path = 'test'#The location where the test images are stored.
for files in os.listdir(img_path):
    imgpath = img_path + '/' + files
    img_pil = Image.open(imgpath)
    input_tensor = test_transform(img_pil).unsqueeze(0).to(device)
    pred_logits = model(input_tensor)
    pred_id = torch.topk(pred_logits, 1)[1].detach().cpu().numpy().squeeze().item()
    activation_map = cam_extractor(pred_id, pred_logits)
    activation_map = activation_map[0][0].detach().cpu().numpy()
    result = overlay_mask(img_pil, Image.fromarray(activation_map), alpha=0.001)
    path2 = '228sscam' + '/' + files #The location where the post-prediction results are stored.
    result.save(path2)

KeyboardInterrupt: 